# Artificial Neural Network

### Importing the libraries

In [134]:
import pandas as pd
import numpy as np 
import tensorflow as tf

In [135]:
tf.__version__

'2.10.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [136]:
df = pd.read_csv("Churn_Modelling.csv")
type(df)

pandas.core.frame.DataFrame

In [137]:
X = df.iloc[:,:-1]
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

In [138]:
type(X)

numpy.ndarray

In [139]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [140]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

In [141]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [142]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


Label Encoding the "Gender" column

One Hot Encoding the "Geography" column

In [143]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [144]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [145]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [146]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

In [147]:
##Hyper Paramaters
HiddenLayerOneSize = 6
HiddenLayerTwoSize = 6
outputSize = 1
activationFunction = 'relu'

### Initializing the ANN

In [148]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [149]:
ann.add(tf.keras.layers.Dense(units=HiddenLayerOneSize,activation=activationFunction))

### Adding the second hidden layer

In [150]:
ann.add(tf.keras.layers.Dense(units=HiddenLayerTwoSize,activation=activationFunction))

### Adding the output layer

In [151]:
ann.add(tf.keras.layers.Dense(units=outputSize,activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [152]:
ann.compile(optimizer = "adam",loss = 'binary_crossentropy',metrics = ['accuracy'])

### Training the ANN on the Training set

In [153]:
ann.fit(X_train,y_train, batch_size = 32, epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 755us/step - loss: 0.6744 - accuracy: 0.6274
Epoch 2/100
250/250 [==============================] - 0s 731us/step - loss: 0.5105 - accuracy: 0.7989
Epoch 3/100
250/250 [==============================] - 0s 735us/step - loss: 0.4571 - accuracy: 0.8096
Epoch 4/100
250/250 [==============================] - 0s 732us/step - loss: 0.4279 - accuracy: 0.8138
Epoch 5/100
250/250 [==============================] - 0s 745us/step - loss: 0.4058 - accuracy: 0.8223
Epoch 6/100
250/250 [==============================] - 0s 737us/step - loss: 0.3879 - accuracy: 0.8364
Epoch 7/100
250/250 [==============================] - 0s 737us/step - loss: 0.3755 - accuracy: 0.8456
Epoch 8/100
250/250 [==============================] - 0s 733us/step - loss: 0.3672 - accuracy: 0.8499
Epoch 9/100
250/250 [==============================] - 0s 729us/step - loss: 0.3614 - accuracy: 0.8516
Epoch 10/100
250/250 [==============================] - 0s 749us/step - l

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [154]:
prediction = ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5

1/1 [==============================] - 0s 51ms/step


In [155]:
prediction

array([[False]])

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [156]:
y_hat = ann.predict(X_test)
y_hat = (y_hat > .5)
print(np.concatenate((y_hat.reshape(len(y_hat),1),y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 563us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [157]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_pred=y_hat,y_true=y_test)
acs = accuracy_score(y_pred=y_hat,y_true=y_test)

In [158]:
cm

array([[1511,   84],
       [ 195,  210]], dtype=int64)

In [159]:
acs

0.8605